# Random Forest with Spark ML

In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

sc = SparkContext.getOrCreate()
ss = SparkSession.builder.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1579223218365_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)
data = sqlContext.read.format('parquet').load("s3://usfca-endomondo/processed_data") 

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data = data.drop("timestamp", "id")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.ml.feature import StringIndexer
def indexStringColumns(df, cols):

    newdf = df
    
    for c in cols:
   
        si = StringIndexer(inputCol=c, outputCol=c+"-num")
        sm = si.fit(newdf)
        newdf = sm.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-num", c)
    return newdf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
data = indexStringColumns(data, ["gender", "sport"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
from pyspark.ml.feature import OneHotEncoder
def oneHotEncodeColumns(df, cols):
    newdf = df
    for c in cols:
        onehotenc = OneHotEncoder(inputCol=c, outputCol=c+"-onehot", dropLast=False)
        newdf = onehotenc.transform(newdf).drop(c)
        newdf = newdf.withColumnRenamed(c+"-onehot", c)
    return newdf

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
data = oneHotEncodeColumns(data, ["gender"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from pyspark.ml.feature import VectorAssembler
input_cols=["miles","avg_heart","var_heart","std_heart","min_alt","max_alt","avg_alt","var_alt","std_alt","gender"]

va = VectorAssembler(outputCol="features", inputCols=input_cols)
data = va.transform(data).select("features", "sport").withColumnRenamed("sport", "label")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
data = data.randomSplit([0.8, 0.2], seed=1)
train = data[0].cache()
test = data[1].cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder

rf = RandomForestClassifier()
evaluator = MulticlassClassificationEvaluator()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [25]).build()
cv = CrossValidator(estimator=rf, evaluator=evaluator, numFolds=5, estimatorParamMaps=paramGrid)
rfmodel = cv.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
test_predicts = rfmodel.transform(test)
metrics =  MulticlassClassificationEvaluator().setLabelCol('label').setPredictionCol('prediction')
metrics.setMetricName("f1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MulticlassClassificationEvaluator_732b9207acbf

In [14]:
train_predicts = rfmodel.transform(train)
print("F1 : %s" % metrics.evaluate(train_predicts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 : 0.9903957406150332

In [13]:
print("F1 : %s" % metrics.evaluate(test_predicts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 : 0.7927766523480106

# Logistic Regression with Spark ML

In [16]:
data_df = ss.read.parquet('s3://usfca-endomondo/processed_data')
data_df = data_df.drop('timestamp', 'id')
from pyspark.ml.feature import StringIndexer

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
data_df = indexStringColumns(data_df, ["gender", "sport"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
data_df = oneHotEncodeColumns(data_df, ["gender"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
label_col = 'sport'
input_cols = data_df.columns
input_cols.remove(label_col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.ml.feature import VectorAssembler
label_col = 'sport'
input_cols = data_df.columns
input_cols.remove(label_col)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
va = VectorAssembler(outputCol="features", inputCols=input_cols)
data_df = va.transform(data_df).select('features', 'sport').withColumnRenamed('sport', 'label')
data_df_train, data_df_test = data_df.randomSplit([0.8, 0.2])
data_df_train.cache()
data_df_test.cache()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[features: vector, label: double]

In [22]:
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
lr = LogisticRegression(regParam=0.01, maxIter=1000, fitIntercept=True)
evaluator = MulticlassClassificationEvaluator()
paramGrid = ParamGridBuilder().addGrid(lr.maxIter, [10, 50, 100, 1000]).addGrid(lr.regParam, [0, 0.01, 0.05, 1]).build()
cv = CrossValidator(estimator=lr, evaluator=evaluator, numFolds=5, estimatorParamMaps=paramGrid)
cvmodel = cv.fit(data_df_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-23:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1199



In [24]:
print("F1 : %s" % MulticlassClassificationEvaluator(metricName='f1').evaluate(cvmodel.bestModel.transform(data_df_test)))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

F1 : 0.711677747642291

In [25]:
cvmodel.bestModel._java_obj.getMaxIter()
cvmodel.bestModel._java_obj.getRegParam()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

0.0

# Deep Learning with H2O

In [26]:
sc.install_pypi_package('h2o-pysparkling-2.4')
sc.install_pypi_package('h2o')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Created wheel for h2o-pysparkling-2.4: filename=h2o_pysparkling_2.4-3.28.0.1.post1-py2.py3-none-any.whl size=101307807 sha256=2daf9500137dac5b3ba2518cbb4a1f1d706614d651ff2e95ca837cfef716453e
  Stored in directory: /var/lib/livy/.cache/pip/wheels/0d/2f/d2/7e0c36c53b57ca597430858ab3d72e1a612a10780016ff6765
  Created wheel for tabulate: filename=tabulate-0.8.6-cp36-none-any.whl size=23271 sha256=e2b943e6eda079ce1848f541a2be2a086d861fb7e60aded9c3ea9a7b6e4df40c
  Stored in directory: /var/lib/livy/.cache/pip/wheels/9c/9b/f4/eb243fdb89676ec00588e8c54bb54360724c06e7fafe95278e
  Created wheel for future: filename=future-0.18.2-cp36-none-any.whl size=491057 sha256=d5a930f9e96bffd35bfccab9a5ad67ebe17663500e4cebb4c9ee701e00afa9ee
  Stored in directory: /var/lib/livy/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
  Created wheel for pyspark: filename=pyspark-2.4.4-py2.py3-none-any.whl size=216130387 sha256=27bd64622f22c26c214af60ed9c5266582b00eeb38c01c83019c0eff6c9

In [32]:
ss = SparkSession.builder.master('yarn').config("spark.dynamicAllocation.enabled", True).getOrCreate()
from pysparkling import *
import h2o
hc = H2OContext.getOrCreate(ss)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


Sparkling Water Context:
 * Sparkling Water Version: 3.28.0.1-1-2.4
 * H2O name: sparkling-water-livy_application_1579223218365_0001
 * cluster size: 2
 * list of used nodes:
  (executorId, host, port)
  ------------------------
  (4,172.31.11.98,54321)
  (1,172.31.0.248,54321)
  ------------------------

  Open H2O Flow in browser: http://ip-172-31-13-122.us-west-2.compute.internal:54321 (CMD + click in Mac OSX)

    
 * Yarn App ID of Spark application: application_1579223218365_0001

In [34]:
data_df = sqlContext.read.format('parquet').load("s3://usfca-endomondo/processed_data") 
data_df = data_df.drop('timestamp', 'id')
data_df_train, data_df_test = data_df.randomSplit([0.8, 0.2])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
data_train_df_h2o = hc.as_h2o_frame(data_df_train, 'train')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
data_df_train_h2o = hc.as_h2o_frame(data_df_train, 'train')
data_df_train_h2o['sport'] = data_df_train_h2o['sport'].asfactor()
data_df_train_h2o['gender'] = data_df_train_h2o['gender'].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
data_df_test_h2o = hc.as_h2o_frame(data_df_test, 'test')
data_df_test_h2o['sport'] = data_df_test_h2o['sport'].asfactor()
data_df_test_h2o['gender'] = data_df_test_h2o['gender'].asfactor()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

dl = H2ODeepLearningEstimator(loss='cross_entropy', nfolds=2, epochs=88, activation="rectifier_with_dropout", balance_classes=False)

VBox()

An error was encountered:
Session 0 did not reach idle status in time. Current status is busy.


### Deep learning works on Shrikar's local laptop, but it took forever to run on EMR. 
### Had a F1=0.674

In [43]:
dl.train(x=input_cols, y=label_col, training_frame=data_df_train_h2o, validation_frame=data_df_test_h2o)

VBox()

An error was encountered:
Session 0 did not reach idle status in time. Current status is busy.


In [ ]:
dl

# Gradient Boosting Trees with H2O

In [52]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

df = sqlContext.read.format('parquet').load("s3://usfca-endomondo/processed_data") 
df_new = df.drop('timestamp', 'id')
# splitting the data
split = df_new.randomSplit([.8, .2])
train = split[0]
valid = split[1]
# turning it into a dataframe
h2o_train = hc.as_h2o_frame(train)
h2o_valid = hc.as_h2o_frame(valid)
# turning the string columns into categorical
h2o_train['gender'] = h2o_train['gender'].asfactor()
h2o_train['sport'] = h2o_train['sport'].asfactor()
h2o_valid['gender'] = h2o_valid['gender'].asfactor()
h2o_valid['sport'] = h2o_valid['sport'].asfactor()
# setup for training
predictors = h2o_train.columns
response = 'sport'
predictors.remove(response)
# GridSearch Gradient Boosting
gbt = H2OGradientBoostingEstimator(nfolds=2, seed=42)
gbt.train(x=predictors, y=response, training_frame=h2o_train, validation_frame=h2o_valid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

gbm Model Build progress: [###############################################] 100%

In [53]:
gbt

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_model_python_1579226151332_5


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 200                         89307                  5            5            5             25            32            30.855


ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.17849465405629125
RMSE: 0.42248627676682143
LogLoss: 0.566470811903318
Mean Per-Class Error: 0.4856787026484119

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
bike    bike (transport)    mountain bike    run    Error     Rate
------  ------------------  ---------------  -----  --------  ----------------
496

# XGBoost with H2O

In [54]:
from h2o.estimators.xgboost import H2OXGBoostEstimator

xgb = H2OXGBoostEstimator(ntrees=20, max_depth=20, nfolds=2, seed=42)
xgb.train(x=predictors, y=response, training_frame=h2o_train, validation_frame=h2o_valid)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

xgboost Model Build progress: [###########################################] 100%

In [55]:
xgb

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_model_python_1579226151332_6


Model Summary: 
    number_of_trees
--  -----------------
    20


ModelMetricsMultinomial: xgboost
** Reported on train data. **

MSE: 0.012129046179988372
RMSE: 0.11013194895210188
LogLoss: 0.07872624706014061
Mean Per-Class Error: 0.005501879121064622

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class
bike    bike (transport)    mountain bike    run    Error        Rate
------  ------------------  ---------------  -----  -----------  -------------
57365   6                   1                104    0.00193124   111 / 57,476
34      6115                0                30     0.0103577    64 / 6,179
41      5                   8544             36     0.00950614   82 / 8,626
12      0                   0                56467  0.000212468  12 / 56,479
57452   6126                8545             56637  0.00208916   269 / 128,760

Top-4 Hit Ratios: 
k    hit_ratio
---